In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Step 1: Load the dataset
# Replace 'your_dataset.csv' with your actual dataset file name
df = pd.read_csv('data/MIZEN_22915.txt', sep="\t", encoding="utf8")
df.columns = ["mi", "en"]



In [ ]:
df['source'] = df['en']
df['target'] = df['mi']
df = df[['source', 'target']]
df.head()

In [ ]:
# Assuming the dataset has columns "source" for English and "target" for Mizo
# Ensure these column names match your dataset

# Step 2: Split the dataset into train and test sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Optional: If you also want a validation set, you can split the train set further
train_df, valid_df = train_test_split(train_df, test_size=0.1, random_state=42)

# Step 3: Save the splits to CSV files
# train_df.to_csv('train.csv', index=False)
# valid_df.to_csv('valid.csv', index=False)  # If you created a validation set
# test_df.to_csv('test.csv', index=False)

print(f"Training set size: {len(train_df)}")
print(f"Validation set size: {len(valid_df)}")
print(f"Test set size: {len(test_df)}")

In [1]:
from datasets import load_dataset, DatasetDict
from transformers import MarianTokenizer
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
# Load your dataset
# Assuming dataset is loaded with columns "source" for English and "target" for Mizo
dataset = load_dataset('csv', data_files={'train': 'data/train.csv', 'validation': 'data/valid.csv'})

# Load tokenizer
# model_name = "Helsinki-NLP/opus-mt-es-mi" #Helsinki-NLP/opus-mt-es-en
# tokenizer = MarianTokenizer.from_pretrained(model_name)

# Load model directly


tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-es-en")
# model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-es-en")

# Tokenize dataset
def preprocess_function(examples):
    inputs = [ex for ex in examples["source"]]
    targets = [ex for ex in examples["target"]]
    model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding="max_length")
    labels = tokenizer(targets, max_length=128, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets = dataset.map(preprocess_function, batched=True)

/home/dt21cs003/workspace/translate/env/lib/python3.10/site-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
/home/dt21cs003/workspace/translate/env/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [3]:
from transformers import MarianMTModel, MarianTokenizer, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = MarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-es-en")


In [4]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,  # Reduce batch size for CPU
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=False,  # No need for FP16 on CPU
    no_cuda=True,  # Ensure we are only using CPU
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    tokenizer=tokenizer,
)

/home/dt21cs003/workspace/translate/env/lib/python3.10/site-packages/transformers/training_args.py:1540: FutureWarning: using `no_cuda` is deprecated and will be removed in version 5.0 of 🤗 Transformers. Use `use_cpu` instead
  warnings.warn(


In [5]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.686400,0.643969
2,0.616900,0.586720
3,0.579800,0.569634


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[65000]], 'forced_eos_token_id': 0}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[65000]], 'forced_eos_token_id': 0}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strate

TrainOutput(global_step=6189, training_loss=0.6744732581696578, metrics={'train_runtime': 14961.2411, 'train_samples_per_second': 3.308, 'train_steps_per_second': 0.414, 'total_flos': 1677663978651648.0, 'train_loss': 0.6744732581696578, 'epoch': 3.0})

In [2]:
with open("data/vanglaini_old_crawled_data.csv", "r", encoding="utf8") as f:
    monolingual_mizo_texts=f.readlines()

## Back translation

In [6]:
from datasets import load_dataset, DatasetDict
from transformers import MarianMTModel, MarianTokenizer, Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
# Load your dataset
# Assuming dataset is loaded with columns "source" for English and "target" for Mizo
# dataset = load_dataset('csv', data_files={'train': 'data/train.csv', 'validation': 'data/valid.csv'})

# Load tokenizer
# model_name = "Helsinki-NLP/opus-mt-es-mi" #Helsinki-NLP/opus-mt-es-en
# tokenizer = MarianTokenizer.from_pretrained(model_name)

# Load model directly

model = MarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-es-en")

tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-es-en")

/home/dt21cs003/workspace/translate/env/lib/python3.10/site-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
/home/dt21cs003/workspace/translate/env/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [10]:
from tqdm import tqdm
import torch

def generate_synthetic_parallel(model, tokenizer, texts, batch_size=4):
    all_translations = []
    with torch.no_grad():  # Disable gradient calculation
        for i in tqdm(range(0, len(texts), batch_size), desc="Back-Translating"):
            batch_texts = texts[i:i + batch_size]
            inputs = tokenizer(batch_texts, return_tensors="pt", padding=True, truncation=True).to("cpu")
            translated = model.generate(**inputs)
            translations = [tokenizer.decode(t, skip_special_tokens=True) for t in translated]
            all_translations.extend(translations)
    return all_translations

# Generate synthetic English texts in small batches with progress bar
synthetic_english_texts = generate_synthetic_parallel(model, tokenizer, monolingual_mizo_texts, batch_size=8)

Back-Translating:   0%|          | 6/14687 [05:31<225:15:23, 55.24s/it]


KeyboardInterrupt: 

### Fine-tuning

In [ ]:
from datasets import DatasetDict

combined_dataset = DatasetDict({
    'train': Dataset.from_dict({
        'source': tokenized_datasets['train']['source'] + synthetic_english_texts,
        'target': tokenized_datasets['train']['target'] + monolingual_mizo_texts,
    }),
    'validation': tokenized_datasets['validation']
})

trainer.train()

### Evaluation

In [ ]:
from datasets import load_metric

metric = load_metric("sacrebleu")

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]

    return metric.compute(predictions=decoded_preds, references=decoded_labels)

results = trainer.evaluate()
print(f"BLEU score: {results['eval_bleu']}")